<table><tr>
<td> <img src="https://upload.wikimedia.org/wikipedia/fr/thumb/e/e5/Logo_%C3%A9cole_des_ponts_paristech.svg/676px-Logo_%C3%A9cole_des_ponts_paristech.svg.png" width="200"  height="200" hspace="200"/> </td>
<td> <img src="https://pbs.twimg.com/profile_images/1156541928193896448/5ihYIbCQ_200x200.png" width="200" height="200" /> </td>
</tr></table>

<br/>

<h1><center>Session 4 - Data preparation</center></h1>



<font size="3">This session is divided into **3** parts:
- **Data exploration**
- **Data preparation**
- **Feature engineering**

In each of these parts, some **guidelines** and **hints** are given for each task. 
Do not hesitate to check the links to documentation to understand the functions you use. 
    
The goal of this session is to **create the dataset** that you will use as an **input for modeling sessions** (supervised and
unsupervised).
</font>

# 0 - Useful libraries and functions

If you do not have the needed libraries already installed please run the following commands in a Terminal:
- pip install pandas
- pip install numpy
- pip install matplotlib
- pip install seaborn
- pip install holidays
- pip install vacances-scolaires-france

In [1]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import holidays
from vacances_scolaires_france import SchoolHolidayDates
%matplotlib inline

In [30]:
# Here are some functions we wrote to help you, feel free to check them out to see what they do
lang_to_keep = ['en', 'fr', 'es', 'it', 'ja', 'de']
country_to_keep = ['FR', 'US', 'GB', 'DE', 'BE', 'CA']
dict_genres = {
    'Drame': 'Drame',
    'Comédie': 'Comédie',
    'Romance': 'Romance',
    'Action': 'Action',
    'Thriller': 'Action',
    'Aventure': 'Action',
    'Crime': 'Action',
    'Guerre': 'Action',
    'Western': 'Action',
    'Familial': 'Familial',
    'Animation': 'Familial',
    'Fantastique': 'Fantastique',
    'Science-Fiction': 'Fantastique',
    'Horreur': 'Horreur',
    'Mystère': 'Other',
    'Musique': 'Other',
    'Histoire': 'Other',
    'Documentaire': 'Other',
    'Téléfilm': 'Other'
}


def read_movies_entrees(path):
    '''
    Read the box office dataset 
    and casts it as an usable pandas DataFrame

    Parameters
    ----------
    path: str
        path to the dataset

    Returns
    -------
    df: pd.DataFrame
        Data as DataFrame
    '''
    bo = read_from_json(path)
    bo = [
        {
            "year": item['year'], 
            "title": item['title'], 
            "id": int(item['id']), 
            "sales": item['first_week_sales'],
            "release_date": item['release_date']
        } for item in bo
    ]
    return pd.DataFrame(bo)


def read_movies_features(path):
    '''
    Read the movie features dataset 
    and casts it as an usable pandas DataFrame
    N.B: Fields that are not yet used are commented
    Parameters
    ----------
    path: str
        path to the dataset
    Returns
    -------
    df: pd.DataFrame
        Data as DataFrame
    '''
    features = read_from_json(path)
    features = [
        {
            "is_adult": item['adult'],
            "is_part_of_collection": not not item['belongs_to_collection'],
            "collection_name": item['belongs_to_collection']['name'] if item['belongs_to_collection'] != {} else None, # Currently simple bool, may be interesting to use a more complex feature later
            "budget": item['budget'],
            "genres": [ genre['name'] for genre in item['genres'] ], 
            "original_language": item['original_language'],
            "overview": item['overview'], # Not used yet. Blob of text
            "production_countries": [ country['iso_code'] for country in item['production_countries'] ],
            "languages": [ language['iso_code'] for language in item['languages'] ],
            "tagline": item['tagline'], # Not used yet. Blob of text
            "runtime": item['runtime'],
            "cast": item['cast'], # Not used yet. List of dicts with actor gender, name, id...
            "id": int(item['id'])
        } for item in features
    ]
    return pd.DataFrame(features)


def read_from_json(path):
    '''
    Read and cast a json into a python object
    
    Parameters
    ----------
    path: str
        Path to json file

    Returns
    -------
    data: Union[dict, list]
        Json casted as python object
    '''
    data = json.load(open(path, 'r', encoding='utf-8', errors="ignore"))
    return data


def reduce_lang_categories(lang_list, lang_to_keep=lang_to_keep):
    return list(set([el if el in lang_to_keep else 'other' for el in lang_list]))


def reduce_country_categories(country_list, country_to_keep=country_to_keep):
    return list(set([el if el in country_to_keep else 'OTHER' for el in country_list]))


def reduce_genre_categories(genre_list, dict_genres=dict_genres):
    return list(set([dict_genres[el] for el in genre_list]))

# 1 - Data exploration

## 1.1 Load data

In [6]:
# We first read the french movies "entrées"
# Hint: you can use the function read_movies_entrees(), you just have to give the path to data as argument
df_boxoffice = read_movies_entrees('../../data/french-box-office-29nov2020.json')

In [31]:
# Then fetch their main features.
# Hint: you can use the function read_movies_features(), you just have to give the path to data as argument
df_features = read_movies_features('../../data/movie-features-29nov2020.json')

In [11]:
df_features.to_csv('movie-features-29nov2020.csv', index=None, sep=';')

In [16]:
df_features[df_features.columns[:-2]].to_csv('movie-features-29nov2020.csv', index=None, sep=';')

In [32]:
df_features

,is_adult,is_part_of_collection,collection_name,budget,genres,original_language,overview,production_countries,languages,tagline,runtime,cast,id
0,False,True,Maléfique - Saga,185000000,"[Fantastique, Familial, Aventure]",en,Cinq années après la conjuration de la malédic...,[US],[en],Aller au-delà du conte de fée.,110.0,"[{'adult': False, 'gender': 1, 'tmdb_id': 1170...",19073
1,False,True,Les Petits Mouchoirs - Saga,0,"[Comédie, Drame]",fr,"Préoccupé, Max est parti dans sa maison au bor...","[BE, FR]",[fr],,135.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 3316...",18875
2,False,True,Spider-Man (Avengers) - Saga,160000000,"[Action, Aventure, Science-Fiction]",en,Peter et ses amis passent leurs vacances d’été...,[US],"[cs, nl, en, de, it]",Il est temps de passer à l'action.,129.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 1136...",18243
3,False,True,Jumanji - Saga,125000000,"[Aventure, Comédie, Fantastique]",en,"L’équipe est de retour, mais le jeu a changé. ...",[US],[en],Bienvenue à Jumanji !,123.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 1891...",18258
4,False,True,Dragons - Saga,129000000,"[Animation, Familial, Aventure]",en,Ce qui avait commencé comme une amitié improba...,"[JP, US]",[en],Une amitié légendaire,104.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 449,...",18167
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7048,False,True,La Planète des Singes - Saga,150000000,"[Drame, Science-Fiction, Guerre]",en,César et les Singes sont contraints de mener u...,"[CA, US, NZ]",[en],Pour l'humanité. Pour l'espoir. Pour la planète.,142.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 1333...",15748
7049,False,True,Cars - Saga,175000000,"[Aventure, Animation, Comédie, Familial]",en,Dépassé par une nouvelle génération de bolides...,[US],[en],La suite des aventures de Flash McQueen,109.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 887,...",16452
7050,False,False,None,17200000,[Comédie],fr,Max est traiteur depuis trente ans. Des fêtes ...,[FR],[fr],,117.0,"[{'adult': False, 'gender': 2, 'tmdb_id': 2828...",17386
7051,False,True,Cinquante nuances - Saga,55000000,"[Drame, Romance]",en,Dépassée par les sombres secrets de Christian ...,[US],[en],,118.0,"[{'adult': False, 'gender': 1, 'tmdb_id': 1185...",15186


In [17]:
temp = pd.read_csv('movie-features-29nov2020.csv', sep=';')
temp

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [ ]:
# Let's merge both dataframes and store them in a new dataframe named "data"
# Hint: you can use the function pd.merge(), you just have to find the right column to merge on (using the "on"
# argument) 
data = 

In [ ]:
# Explore the first lines of your dataset. Feel free to do that every time you implement a new feature
data.head(10)

## 1.2 EDA on sales

In [ ]:
# Check the characteristics of the feature "sales" of the dataset using statistics
# Hint: you can use the method .describe() on your dataframe on the right column

In [ ]:
# Check the characteristics of the feature "sales" of the dataset using graph
# Hint: we advise you to use seaborn. Take a look at the example gallery and choose a graph that you think is 
# relevant to answer the question (https://seaborn.pydata.org/examples/index.html)
# NB: if you want to scale your graph in order to make it readable add the following arguments in your function:
# height=5, aspect=11/8 (ex: sns.lineplot(x=x, data=data, height=5, aspect=11/8))

In [ ]:
# Is there any outlier in the dataset ? (e.g.: a movie with no sales)
# Explain how you would like to handle this issue?

## 1.3 EDA on other features

In [ ]:
# Explore other features of the dataset and keep note of your findings, it can help for the next parts
# Hint: you can use .info() on your dataframe to gather information on variables types

In [ ]:
# Plot the distribution of movies across years. Fill the arguments: x, data, kind below
# Hint: we advise you to use seaborn. Take a look at the example gallery and choose a graph that you think is 
# relevant to answer the question (https://seaborn.pydata.org/examples/index.html)
# NB: if you want to scale your graph in order to make it readable add the following arguments in your function:
# height=5, aspect=11/8 (ex: sns.lineplot(x=x, data=data, height=5, aspect=11/8))

In [ ]:
# Plot the distribution of the "budget" feature
# Hint: you can use sns.displot() for continuous variables or sns.catplot() for categorical variables. Choose well !

In [ ]:
# Plot the distribution of the "is_part_of_collection" feature
# Hint: you can use sns.displot() for continuous variables or sns.catplot() for categorical variables. Choose well !

In [ ]:
# Plot the distribution of the "runtime" feature
# Hint: you can use sns.displot() for continuous variables or sns.catplot() for categorical variables. Choose well !

In [ ]:
# For genres, languages, production countries, data comes in list for each movie, 
# we should preprocess these columns before plotting them. Here we flatten the lists into a bigger Series

def flatten_list_series(column):
    flattened_series = column.apply(pd.Series).stack().reset_index(drop=True)
    flattened_series.name = column.name
    return pd.DataFrame(flattened_series)

# Try to plot a histogram for these categorical features using sns.catplot() with the following argument:
# data=flatten_list_series(data['your_column'])

> **To go further:**
Explore the **"cast"** feature: 
 - What insights can you find? 
 - What kind of difficulties can you anticipate? 
 - How do you think we can use this feature for our model later?


# 2 - Data preparation

## 2.1 Missing values

In [ ]:
# Missing/Zero values for sales
# Hint: check the number of missing values to see which approach is the best suited

In [ ]:
# Missing/zero values for budget
# Hint: check the number of missing values to see which approach is the best suited

In [ ]:
# Missing/zero values for runtime
# Hint: check the number of missing values to see which approach is the best suited

In [ ]:
# To go further:
# Missing values for production countries
# Hint: you can use the column "original_language" (e.g.: if original_language is "FR", the production country
# is likely to be France)

## 2.2 Reduce cardinality for categorical features

In [ ]:
# Reduce number of categories for: Original language
# Hint: you can use the lang_to_keep variable from part 0, already imported. If the original language is in
# lang_to_keep then we keep it, otherwise we set the value to 'other'
data['original_language'] = data['original_language'].map(lambda x: x if x in ... else ...)

In [ ]:
# Reduce number of categories for: languages, production_countries and genres
# For example for languages:
data['languages'] = data['languages'].map(lambda x: reduce_lang_categories(x))

# Hint: for production_countries, you can use the reduce_country_categories() function from part 0,
# already imported
data['production_countries'] = 

# Hint: for production_countries, you can use the reduce_genre_categories() function from part 0,
# already imported
data['genres'] = 

## 2.3 Encoding

In [ ]:
# Encode is_part_of_collection into numerical
# Hint: you can use a dictionary to map numerical values for each value of is_part_of_collection (True or False)
data['is_part_of_collection'] = 

In [ ]:
# Encode original_language
# Hint: you can perform one-hot encoding with pd.get_dummies(), check documentation to see how to use it (
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)
data_final = 

In [ ]:
# Encode categorical features with multiple categories
# Hint: you can use MultiLabelBinarizer()

# Languages
mlb = MultiLabelBinarizer()
df_lang = pd.DataFrame(mlb.fit_transform(data_final['languages']), columns=mlb.classes_, index=data_final.index)
df_lang.columns = ['available_lang_' + col for col in df_lang.columns]

# Genres
mlb = MultiLabelBinarizer()
df_genre = 

# Production countries
mlb = MultiLabelBinarizer()
df_country = 

In [ ]:
# Merge encoded dataframes and store it into a new dataframe named data_final
# Hint: you can use the merge function from pandas, pd.merge(), check documentation to see how to use it
data_final = 

# 3 - Feature engineering

## 3.1 Holidays

In [ ]:
# Load school holidays for France
fr_holidays = SchoolHolidayDates()
df_vacances = pd.DataFrame()
for year in list(set(data_final['year'])):
    df_vacances = pd.concat([df_vacances, pd.DataFrame.from_dict(fr_holidays.holidays_for_year(year)).T])

# Load bank holidays for France
df_jf = pd.DataFrame()
for year in list(set(data_final['year'])):
    df_jf = pd.concat([df_jf, pd.DataFrame([
        {'date': el[0], 'jour_ferie': el[1]} for el in sorted(holidays.FRA(years=year).items())])])
    
# Merge school and bank holidays
df_holidays = pd.merge(df_vacances, df_jf, how='outer', on='date')

In [ ]:
# Create features from df_holidays dataframes (school holidays and bank holidays):
# - 3 binary features for school holidays, taking 1 if the given zone is on holiday, else 0 (vacances_zone_a, 
# vacances_zone_b, vacances_zone_c)
# - 1 binary feature for bank holiday, taking 1 if it is a bank holiday, else 0
# - To go further: Try to create a combined feature with school and bank holidays


In [ ]:
# Merge df_holidays that contains newly created features with the main dataframe and store it into data_final
df_holidays['date'] = df_holidays['date'].map(lambda x: str(x))
data_final = pd.merge(data_final, df_holidays, how='left', left_on='release_date', right_on='date').fillna(0)

## 3.2 Calendar

In [ ]:
# Create calendar features for month: 
# - the number of the month (named "month")

# To go further: try to transform the "month" variable using a mathematical function to capture cyclicity
# (January (1) comes right after December (12))

## 3.3 Collection

In [ ]:
# Collection with an high number of movies are often sagas that have worked well (ex: Star Wars, Fast and
# Furious, ...)
# We can therefore use the variable "is_part_of_collection" to compute the number of movies per collection
# Hint: to create this kind of feature, you can use the .value_counts() method

In [ ]:
# To go (much) further:
# Movies from a same collection will tend to have a similar number of sales
# We can therefore use the variable "is_part_of_collection" to calculate an average of the sales of previous films
# from the same collection
# Hint: to create this kind of feature, you can use the .groupby(), .transform(), .rolling(), .mean() and .shift()
# methods

## 3.4 [OPTIONAL] Casting

In [ ]:
# Explore the "cast" feature to understand its structure and try to identify its limits if not done in Part 1.

In [ ]:
# A movie with bankable actors is more likely to have an important number of entrees than a movie with unknown 
# actors
# We can leverage the "cast" feature that contains information about actors and their popularity for each movie
# to create several features, for example:
# - for one movie, compute the mean popularity of its 3 main actors
# - for one movie, compute the mean popularity of its 5 main actors

In [ ]:
# To go (much much) further:
# In the same vein, we could create features taking into account sales of previous movies per actor and create 
# features that represent:
# - for one movie, the mean of sales of previous movies of the #1 actor
# - for one movie, the mean of sales of previous movies of the #2 actor
# - for one movie, the mean of sales of previous movies of the #3 actor
# - for one movie, the mean or the maximum of the three features above
# This would also give an idea of an actor's "popularity"

# 4 - Save file for next session

In [ ]:
# Drop useless columns for modeling (not numerical columns, raw columns that have been transformed, ...)
# Hint: you can use the .drop() method


In [ ]:
# Fill missing values due to feature engineering if any
# Hint: you can use the .fillna() method


In [ ]:
# Save file
# Hint: you can use the .to_csv() method
